# Return Predictions Using SEC Filings Embeddings 

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import os
from pathlib import Path
from time import time
from collections import defaultdict
from datetime import datetime, timedelta
from tqdm import tqdm
import logging
import h5py

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

import matplotlib.pyplot as plt
import seaborn as sns

import datasets

from gensim.models import KeyedVectors, Doc2Vec, doc2vec
from gensim.utils import simple_preprocess

from sklearn.model_selection import train_test_split

import lightgbm as lgb

In [106]:
NUM_PROC = 4
BATCH_SIZE = 128

sec_path = Path('sec-edgar-10k')

# results_path = Path('results', 'sec-filings')
model_path = sec_path / 'models'
log_path = sec_path / 'logs'
data_path = sec_path / 'data'
parsed_data_path = sec_path / 'parsed-data'

logging.basicConfig(
    filename=log_path / 'lgbm.log',
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)

## Prepare Model Data

### Get Stock Prices and SEC Filing Dates

In [3]:
filing_index = pd.read_csv(sec_path / 'filing_index.csv', parse_dates=['DATE_FILED']).rename(columns=str.lower)
filing_index.index += 1   # add 1 to match filing index with filings in price data 

In [4]:
filing_index.head()

,cik,company_name,form_type,date_filed,edgar_link,quarter,ticker,sic,exchange,hits,year
1,1000180,SANDISK CORP,10-K,2013-02-19,edgar/data/1000180/0001000180-13-000009.txt,1,SNDK,3572,NASDAQ,3,2013
2,1000209,MEDALLION FINANCIAL CORP,10-K,2013-03-13,edgar/data/1000209/0001193125-13-103504.txt,1,TAXI,6199,NASDAQ,0,2013
3,1000228,HENRY SCHEIN INC,10-K,2013-02-13,edgar/data/1000228/0001000228-13-000010.txt,1,HSIC,5047,NASDAQ,3,2013
4,1000229,CORE LABORATORIES N V,10-K,2013-02-19,edgar/data/1000229/0001000229-13-000009.txt,1,CLB,1389,NYSE,2,2013
5,1000232,KENTUCKY BANCSHARES INC KY,10-K,2013-03-28,edgar/data/1000232/0001104659-13-025094.txt,1,KTYB,6022,OTC,0,2013


### Create weekly forward returns

We will compute (somewhat arbitrarily) five-day forward returns for the day of filing (or the day before if there are no prices for that date), assuming that filing occurred after market hours. Clearly, this assumption could be wrong, underscoring the need for **point-in-time data**.

In [5]:
prices = pd.read_hdf(sec_path / 'sec_returns.h5', 'prices')
prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1228824 entries, 2013-09-17 00:00:00-04:00 to 2015-01-23 00:00:00
Data columns (total 8 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   filing  1228824 non-null  int64  
 1   ticker  1228824 non-null  object 
 2   cik     1228824 non-null  int64  
 3   open    1228802 non-null  float64
 4   high    1228820 non-null  float64
 5   low     1228820 non-null  float64
 6   close   1228821 non-null  float64
 7   volume  1228824 non-null  float64
dtypes: float64(5), int64(2), object(1)
memory usage: 84.4+ MB


In [6]:
filings = prices.filing.unique()

We compute the forward returns as follows, removing outliers with weekly returns below
50 or above 100 percent:

In [7]:
years = set()
fwd_return = {}

for filing in filings:
    date_filed = filing_index.at[filing, 'date_filed']
    price_data = prices[prices.filing==filing].close.sort_index()
    date_filed = date_filed.tz_localize(price_data.index[0].tz)
    cik = filing_index.at[filing, 'cik']
    year = date_filed.year
    years.add(year)
    
    try:
        r = (price_data
             .pct_change(periods=5)
             .shift(-5)
             .loc[:date_filed]
             .iloc[-1])
    except:
        continue
    if not np.isnan(r) and -.5 < r < 1:
        fwd_return[(str(cik), str(year))] = r

In [8]:
multi_index = pd.MultiIndex.from_tuples(list(fwd_return.keys()), names=['cik', 'year'])

In [9]:
returns = pd.DataFrame([], index=multi_index, columns=['returns'])

In [10]:
for k, v in fwd_return.items():
    returns.loc[k, 'returns'] = v

In [11]:
CIK_YEAR = returns.index.to_list()
years = sorted(list(years))

## Train Gradient Boosting Model with SEC Document Embeddings from Trained Gensim Model

We will use the embedding model (named `doc2vec_1.model`) that has been trained in the first notebook using Gensim to generate the doc-vectors of SEC filings for train and test data as follows.

In [12]:
print(f"Filter and process SEC filings data for {years[:-1]} as train and {years[-1]} as test data.")
TEST_YEAR = years[-1]

Filter and process SEC filings data for [2013, 2014, 2015] as train and 2016 as test data.


In [13]:
if os.path.exists(parsed_data_path):
    dataset = datasets.load_from_disk(parsed_data_path.as_posix())

train_parsed = dataset['train']

In [14]:
if os.path.exists(model_path / 'doc2vec_1.model'):
    gensim_model = Doc2Vec.load((model_path / 'doc2vec_1.model').as_posix())

In [15]:
COL_NAMES = set(train_parsed.column_names) - set(['cik', 'year'])

In [16]:
# keep only those companies for which we have price data from 2013-2016
train_parsed = train_parsed.filter(lambda examples: (examples['cik'], examples['year']) in CIK_YEAR, num_proc=NUM_PROC)

In [17]:
def get_doc2vec_embeddings(examples, model, col_name):
    ciks = []
    years = []
    items = []
    
    for doc, cik, year in zip(examples[col_name], examples['cik'], examples['year']):
        tokens = simple_preprocess(doc)
        vector = model.infer_vector(tokens)
        ciks += [cik]
        years += [year]
        items += [vector]

    return {'cik': ciks, 'year': years, f"{col_name}_vec": items}

In [18]:
def process_to_pandas(model, data, select_size=None):
    dfs = []
    if select_size:
        data = data.select(range(select_size))

    for col in COL_NAMES:
        dfs += [
            data.map(
                get_doc2vec_embeddings, batched=True, batch_size=BATCH_SIZE, num_proc=NUM_PROC, remove_columns=data.column_names, fn_kwargs={'model': model, 'col_name': col}
            ).to_pandas()
        ]

    final = dfs[0][['cik' , 'year']].set_index(['cik' , 'year'], drop=True)
    for df in dfs:
        df.set_index(['cik' , 'year'], drop=True, inplace=True)
        final = final.join(df, how='inner')

    return final

In [19]:
train_val_test = process_to_pandas(gensim_model, train_parsed, select_size=None)

Map (num_proc=4):   0%|          | 0/14133 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/14133 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/14133 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/14133 [00:00<?, ? examples/s]

### Combine returns with filings doc vectors

In [20]:
train_val_test = train_val_test.join(returns, how='inner')
train_val_test.to_hdf(sec_path / 'sec_returns.h5', 'data/doc_vecs_return')

### Split data and train a LightGBM model

In [328]:
train_val_test = pd.read_hdf(sec_path / 'sec_returns.h5', 'data/doc_vecs_return')

In [335]:
def prepare_data_for_training(data, features, constraint_features):
    df = data.reset_index()
    constraints = []
    
    if not constraint_features:
        # stack all SEC sections in one column and add section number as a categorical feature
        # total number of features = embedding size + section number (384 + 1)
        temp = pd.DataFrame([], columns=['cik', 'year', 'returns', 'features', 'section'])
        for col in features:
            df_sub = df[['cik', 'year', 'returns', col]].rename(columns={col: 'features'})
            df_sub['section'] = col.split('_')[2] if 'section' not in df.columns else df['section']
            temp = pd.concat([temp, df_sub], axis=0, ignore_index=True)
        embedding_size = df[col].iloc[0].shape[0]
        categories = list(temp.section.astype('category').unique())
        temp['year'] = temp['year'].astype(np.int32)
        temp['returns'] = temp['returns'].astype(np.float32)
        temp['section'] = temp['section'].astype('category').cat.codes
        temp['features'] = temp[['features', 'section']].apply(lambda x: np.append(x['features'], x['section']), axis=1)
        train_val = temp.loc[temp.year != TEST_YEAR]
        test = temp.loc[temp.year == TEST_YEAR]
        cat_idx = [embedding_size]
    else:
        # concatanate all section embeddings horizontally and add constraint for each section within the feature
        # vectors. Also add section number as a categorical feature
        # total number of features = (number of sections) * (embedding size) + section number (4*384 + 1)
        df['features'] = df[features].apply(lambda x: np.stack([x.values[i] for i in range(x.values.shape[0])]).reshape(-1, 1).squeeze(), axis=1)
        for col in features:
            df['section'] = col.split('_')[2]
        embedding_size = df[col].iloc[0].shape[0]
        categories = list(df.section.astype('category').unique())
        df['year'] = df['year'].astype(np.int32)
        df['returns'] = df['returns'].astype(np.float32)
        df['section'] = df['section'].astype('category').cat.codes
        df['features'] = df[['features', 'section']].apply(lambda x: np.append(x['features'], x['section']), axis=1)
        train_val = df.loc[df.year != TEST_YEAR]
        test = df.loc[df.year == TEST_YEAR]
        constraints = [list(range(i*embedding_size, (i+1)*embedding_size)) + [embedding_size*len(features)] for i in range(len(features))]
        cat_idx = [embedding_size*len(features)]

    # split train data into 90-10 train-validation sets
    X_train, X_val, y_train, y_val = train_test_split(np.stack(train_val['features'].tolist()), train_val['returns'].values, test_size=0.1)
    X_test, y_test = np.stack(test['features'].tolist()), test['returns'].values

    train_data = lgb.Dataset(data=X_train, label=y_train, categorical_feature=cat_idx, free_raw_data=False)
    valid_data = train_data.create_valid(data=X_val, label=y_val)
    
    return (train_data, valid_data), (X_test, y_test), constraints

In [330]:
def train_booster_and_test(data, params, constraint_features=False):
    (train, valid), test, constraints = prepare_data_for_training(data, [col + "_vec" for col in COL_NAMES], constraint_features)

    params['interaction_constraints'] = constraints

    bst = lgb.train(
        params=params,
        train_set=train,
        valid_sets=valid,
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )
    if constraint_features:
        bst.save_model(model_path / 'lgb_model_with_constraints.txt', num_iteration=bst.best_iteration)
    else:
        bst.save_model(model_path / 'lgb_model_no_constraints.txt', num_iteration=bst.best_iteration)
    print("Model training is done.")

    y_score = bst.predict(test[0], num_iteration=bst.best_iteration)
    rho, p = spearmanr(y_score, test[1])
    print(f"\nTesting the model on {TEST_YEAR} data:")
    print(f"Spearman rank corr={rho:.2%} (p-value={p:.3f})")

In [336]:
params = {
        'objective': 'huber',
        'boosting': 'gbdt',  # gbdt or dart
        'drop_rate': 0.1,    # dropout rate in dart
        'num_boost_round': 1000,
        'learning_rate': 0.01,
        'num_leaves': 25,
        'num_threads': 4,
        'bagging_fraction': 0.8, # fraction for subsampling data (<=1)
        'bagging_freq': 1, # 0 means no bagging
        'alpha': 0.9,   # huber loss parameter
        'metric': ['mae', 'mse', 'huber'],
        'min_data_in_leaf': 1,
        'force_col_wise': True,
    }

In [337]:
train_booster_and_test(train_val_test, params, constraint_features=False)

[LightGBM] [Info] Total Bins 97925
[LightGBM] [Info] Number of data points in the train set: 38822, number of used features: 385
[LightGBM] [Info] Start training from score 0.005824
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[26]	valid_0's l1: 0.0419229	valid_0's l2: 0.00533583	valid_0's huber: 0.00266792
Model training is done.

Testing the model on 2016 data:
Spearman rank corr=2.69% (p-value=0.002)


In [340]:
train_booster_and_test(train_val_test, params, constraint_features=True)

[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 9705, number of used features: 1536
[LightGBM] [Info] Start training from score 0.005892
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[66]	valid_0's l1: 0.0398812	valid_0's l2: 0.00427291	valid_0's huber: 0.00213646
Model training is done.

Testing the model on 2016 data:
Spearman rank corr=2.95% (p-value=0.088)


The above results suggest that **Information Coeffecient** is not highly significant when we train the booster on different sections (p-value=0.088). This is mainly due to smaller train set size (10k vs. 40k doc vectors) and more number of features (1536 vs. 385). More training data is needed to attain a clear conclusion on which training logic performs better.  

---

## Train Gradient Boosting Model with SEC Document Embeddings from Sentence Transformer

In [107]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

# this model is fine tuned on nreimers/MiniLM-L6-H384-uncased pre-trained model
model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)
sbert_model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

MAX_LENGTH = tokenizer.max_len_single_sentence
BATCH_SIZE = 16
SECTIONS = ['section_1', 'section_1A', 'section_7', 'section_7A']

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [206]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask, sample_map):
    batch_result = []
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = tf.expand_dims(tf.cast(attention_mask, tf.float32), -1)
    # Perform mean pooling over chunks of each sequence
    for i in range(tf.reduce_max(sample_map) + 1):
        s_idx = np.where(sample_map==i)[0][0]
        e_idx = np.where(sample_map==i)[0][-1] + 1
        sum_embeddings = tf.reduce_sum(token_embeddings[s_idx:e_idx] * input_mask_expanded[s_idx:e_idx], axis=[0, 1])
        sum_mask = tf.clip_by_value(tf.reduce_sum(input_mask_expanded[s_idx:e_idx]), clip_value_min=1e-7, clip_value_max=np.inf)
        batch_result += [sum_embeddings / sum_mask]
        
    return tf.stack(batch_result)

def get_transformer_embeddings(data, col_name):
    encoded_input = tokenizer(
        data[col_name],
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,  # Large sequences are chunked into arrays of max_length
        return_tensors="tf"
    )

    # Identifies what sequence chunk belongs to which sample
    sample_map = encoded_input.pop("overflow_to_sample_mapping")
    attention_mask = encoded_input["attention_mask"]
    model_output = sbert_model(**encoded_input)
    
    return {
        "cik": data["cik"],
        "year": data["year"],
        "section": data["section"],
        f"{col_name}_embedding": mean_pooling(model_output, attention_mask, sample_map),
    }

In [190]:
dataset = datasets.load_from_disk(data_path.as_posix())
train = datasets.concatenate_datasets([dataset["train"], dataset["validation"]], axis=0)
# keep only those companies for which we have price data from 2013-2016
train = train.filter(lambda examples: (examples['cik'], examples['year']) in CIK_YEAR, num_proc=NUM_PROC)

test = train.filter(lambda row: int(row['year']) == TEST_YEAR)
train = train.filter(lambda row: int(row['year']) in years[:-1])

In [191]:
# select a subset of 750 companies to speedup embeddings generation on a small machine
SUBSET_SIZE = 750
subset = np.random.choice(np.array(list(set(train['cik']).intersection(set(test['cik'])))), size=SUBSET_SIZE, replace=False)

In [198]:
def select_subset_and_wrap(data, subset, col_names, new_name):
    data = data.filter(lambda row: row['cik'] in subset)
    
    wrapped = []
    for col in col_names:
        temp = data.select_columns(['cik', 'year', col])
        temp = temp.filter(lambda example: example[col] != '')
        temp = temp.add_column(name='section',  column=[col.split('_')[-1]] * len(temp))
        temp = temp.rename_column(col, new_name)
        wrapped.append(temp)

    return wrapped

In [199]:
# Concatenate all sections of raw SEC filings in one column to be fed to the transformer model
wrapped = select_subset_and_wrap(train, subset, SECTIONS, new_name='raw_sections')
train = datasets.concatenate_datasets(wrapped, axis=0)

wrapped = select_subset_and_wrap(test, subset, SECTIONS, new_name='raw_sections')
test = datasets.concatenate_datasets(wrapped, axis=0)

Filter:   0%|          | 0/10784 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2124 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1971 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2124 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1973 [00:00<?, ? examples/s]

...

Flattening the indices:   0%|          | 0/694 [00:00<?, ? examples/s]

Filter:   0%|          | 0/750 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/730 [00:00<?, ? examples/s]

Filter:   0%|          | 0/750 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/717 [00:00<?, ? examples/s]

In [211]:
train_embeddings = train.map(get_transformer_embeddings, batched=True, batch_size=BATCH_SIZE, remove_columns=test.column_names, fn_kwargs={'col_name': 'raw_sections'})
test_embeddings = test.map(get_transformer_embeddings, batched=True, batch_size=BATCH_SIZE, remove_columns=test.column_names, fn_kwargs={'col_name': 'raw_sections'})

Map:   0%|          | 0/8049 [00:00<?, ? examples/s]

Map:   0%|          | 0/2834 [00:00<?, ? examples/s]

In [212]:
train_val_test_transformer = pd.concat([
    train_embeddings.to_pandas().set_index(['cik' , 'year'], drop=True).join(returns),
    test_embeddings.to_pandas().set_index(['cik' , 'year'], drop=True).join(returns)
], axis=0)

In [214]:
train_val_test_transformer.to_hdf(sec_path / 'sec_returns.h5', 'data/doc_vecs_return_transformer')

### Split data and train a LightGBM model

In [341]:
train_val_test_transformer = pd.read_hdf(sec_path / 'sec_returns.h5', 'data/doc_vecs_return_transformer')

In [342]:
def train_booster_and_test_for_transformer(data, params):
    (train, valid), test, constraints = prepare_data_for_training(data, ["raw_sections_embedding"], constraint_features=False)

    params['interaction_constraints'] = constraints

    bst = lgb.train(
        params=params,
        train_set=train,
        valid_sets=valid,
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )
    
    bst.save_model(model_path / 'lgb_model_for_transformer.txt', num_iteration=bst.best_iteration)
    print("Model training is done.")

    y_score = bst.predict(test[0], num_iteration=bst.best_iteration)
    rho, p = spearmanr(y_score, test[1])
    print(f"\nTesting the model on {TEST_YEAR} data:")
    print(f"Spearman rank corr={rho:.2%} (p-value={p:.3f})")

In [343]:
params = {
        'objective': 'huber',
        'boosting': 'gbdt',  # or dart
        'drop_rate': 0.1,    # dropout rate in dart
        'num_boost_round': 1000,
        'learning_rate': 0.01,
        'num_leaves': 25,
        'num_threads': 4,
        'bagging_fraction': 0.8, # fraction for subsampling data (<=1)
        'bagging_freq': 1, # 0 means no bagging
        'alpha': 0.9,   # huber loss parameter
        'metric': ['mae', 'mse', 'huber'],
        'min_data_in_leaf': 1,
        'force_col_wise': True,
    }

In [344]:
train_booster_and_test_for_transformer(train_val_test_transformer, params)

[LightGBM] [Info] Total Bins 97925
[LightGBM] [Info] Number of data points in the train set: 7244, number of used features: 385
[LightGBM] [Info] Start training from score 0.006355
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's l1: 0.0400413	valid_0's l2: 0.00481805	valid_0's huber: 0.00240902
Model training is done.

Testing the model on 2016 data:
Spearman rank corr=-2.34% (p-value=0.213)


High p-value for information coefficient here again suggests that the train set size is not large enough to draw any conclusion. This is reasonable as we have selected a subset of 750 companies to generate the embedding vectors due to demanding computations.